As a prior step please set ANALYZE_DATASET = True in dp6_dataset.py

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import shutil

# importing the sys module
import sys        
 
# appending the directory of mod.py
# in the sys.path list
sys.path.append('../')   

import json
import os
import random
from collections import defaultdict

import torch
from PIL import Image
from torch import nn
from torch.optim.lr_scheduler import CosineAnnealingLR
from torchvision import models
from torch.utils.data import DataLoader, WeightedRandomSampler
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
from torch.optim import lr_scheduler
from torch.optim import Adam
import numpy as np
from sklearn.model_selection import train_test_split

from datetime import datetime

from Common.Optimizers.RangerLars import RangerLars
from Common.Schedulers.GradualWarmupScheduler import GradualWarmupScheduler

import dp6_config as cfg

from dp6_dataset import AgeDiffSameUniformDiffDataset, AgeDiffMixedUniformDiffDataset, AgeDiffMimicDiffDataset, get_error_constrained_dataset
from Common.Datasets.Morph2.data_parser import DataParser
from dp6_train import train_diff_cls_model_iter


from dp6_model import AgeDiffModel, DiffModelConfigType

from tqdm import tqdm

from sklearn import utils



#####################################################
#           Preparations
#####################################################

torch.manual_seed(cfg.RANDOM_SEED)
np.random.seed(cfg.RANDOM_SEED)
random.seed(cfg.RANDOM_SEED)

if cfg.USE_GPU:
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
else:
    device = torch.device("cpu")

print(device)

torch.cuda.empty_cache()

#####################################################
#           Data Loading
#####################################################

with open("im2age_map_test.json", 'r') as im2age_map_test_f:
    im2age_map_test = json.load(im2age_map_test_f)

# Load data
data_parser = DataParser('../Common/Datasets/Morph2/aligned_data/aligned_dataset_with_metadata_uint8.hdf5', small_data=cfg.SMALL_DATA)
data_parser.initialize_data()


x_train, y_train, x_test, y_test = data_parser.x_train,	data_parser.y_train, data_parser.x_test, data_parser.y_test,
if cfg.RANDOM_SPLIT:
    all_images = np.concatenate((x_train, x_test), axis=0)
    all_labels = np.concatenate((y_train, y_test), axis=0)

    x_train, x_test, y_train, y_test = train_test_split(all_images, all_labels, test_size=cfg.TEST_SIZE_FOR_RS, random_state=cfg.RANDOM_SEED)


cuda:0


In [4]:
face2emb_arr_trn_r = np.load('face2emb_arr_trn.npy', allow_pickle=True)
#     import pdb
#     pdb.set_trace()
#face2emb_arr_trn_r = face2emb_arr_trn_r.item()

face2emb_arr_vld_r = np.load('face2emb_arr_vld.npy', allow_pickle=True)
#face2emb_arr_vld_r = face2emb_arr_vld_r.item()

# shuffle basic aligned test
# x_test_shuffled
# y_test_shuffled


transf = transforms.Compose([
        transforms.RandomResizedCrop(224, (0.9, 1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomApply([transforms.ColorJitter(
            brightness=0.1,
            contrast=0.1,
            saturation=0.1,
            hue=0.1
        )], p=0.5),
        transforms.RandomApply([transforms.RandomAffine(
            degrees=10,
            translate=(0.1, 0.1),
            scale=(0.9, 1.1),
            shear=5,
            resample=Image.BICUBIC
        )], p=0.5),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        transforms.RandomErasing(p=0.5)
    ])

# transf = transforms.Compose([
#         transforms.Resize(224), # just for testing
# #          transforms.RandomResizedCrop(224, (0.9, 1.0)),
# #          transforms.RandomHorizontalFlip(),
#         transforms.RandomApply([transforms.ColorJitter(
#             brightness=0.1,
#             contrast=0.1,
#             saturation=0.1,
#             hue=0.1
#         )], p=0.5),
# #         transforms.RandomApply([transforms.RandomAffine(
# #             degrees=10,
# #             translate=(0.1, 0.1),
# #             scale=(0.9, 1.1),
# #             shear=5,
# #             resample=Image.BICUBIC
# #         )], p=0.5),
#         transforms.ToTensor(),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
# #        transforms.RandomErasing(p=0.5)
#     ])


train_ds = AgeDiffSameUniformDiffDataset(
    data_set_images=x_train,
    data_set_metadata=y_train,
    min_age=cfg.MIN_AGE,
    age_interval=cfg.AGE_INTERVAL,
    max_age=cfg.MAX_AGE,
    transform=transf,
    copies=1, 
    age_diff_learn_radius_lo=cfg.AGE_DIFF_LEARN_RADIUS_LO,
    age_diff_learn_radius_hi=cfg.AGE_DIFF_LEARN_RADIUS_HI,
    embs=face2emb_arr_trn_r,
    num_references=cfg.NUM_OF_REFS,
    use_embs=cfg.USE_EMBEDDINGS,
    embs_knn=cfg.EMBEDDINGS_KNN
)

print("Training (q trn, r trn) set size: " + str(len(train_ds)))

test_qtst_rtst_ds = AgeDiffSameUniformDiffDataset(
    data_set_images=x_test,
    data_set_metadata=y_test,
    min_age=cfg.MIN_AGE,
    age_interval=cfg.AGE_INTERVAL,
    max_age=cfg.MAX_AGE,
    transform=transf,
    copies=1, 
    age_diff_learn_radius_lo=cfg.AGE_DIFF_LEARN_RADIUS_LO,
    age_diff_learn_radius_hi=cfg.AGE_DIFF_LEARN_RADIUS_HI,
    embs=face2emb_arr_vld_r,
    num_references=cfg.NUM_OF_REFS,
    use_embs=cfg.USE_EMBEDDINGS,
    embs_knn=cfg.EMBEDDINGS_KNN
)

print("Testing (q tst, r tst) set size: " + str(len(test_qtst_rtst_ds)))

test_qtst_rtrn_ds = AgeDiffMixedUniformDiffDataset(
    batrn_set_images=x_train,
    batrn_set_metadata=y_train,
    batst_set_images=x_test,
    batst_set_metadata=y_test,
    min_age=cfg.MIN_AGE,
    age_interval=cfg.AGE_INTERVAL,
    max_age=cfg.MAX_AGE,
    transform=transf,
    copies=1, 
    age_diff_learn_radius_lo=cfg.AGE_DIFF_LEARN_RADIUS_LO,
    age_diff_learn_radius_hi=cfg.AGE_DIFF_LEARN_RADIUS_HI,
    embs_trn=face2emb_arr_trn_r,
    embs_vld=face2emb_arr_vld_r,
    num_references=cfg.NUM_OF_REFS,
    use_embs=cfg.USE_EMBEDDINGS,
    embs_knn=cfg.EMBEDDINGS_KNN
)

print("Testing (q tst, r trn) set size: " + str(len(test_qtst_rtrn_ds)))

x_test_filtered, y_test_filtered, batst_set_filtered_indexes = get_error_constrained_dataset(orig_dataset_images=x_test, 
                                                                                                orig_dataset_metadata=y_test,
                                                                                                age_diff_learn_radius_lo=cfg.APREF_VLD_SET_AGE_DIFF_LEARN_RADIUS_LO,
                                                                                                age_diff_learn_radius_hi=cfg.APREF_VLD_SET_AGE_DIFF_LEARN_RADIUS_HI,#cfg.AGE_DIFF_LEARN_RADIUS_HI,
                                                                                                im2age_map_batst=im2age_map_test)

# apref = Age Predict Reference (dataset is based on (q,r) pairs where r's age is what AgePredict model returns on q)
test_apref_ds = AgeDiffMimicDiffDataset(
    batrn_set_images=x_train,
    batrn_set_metadata=y_train,
    batst_set_images=x_test_filtered,
    batst_set_metadata=y_test_filtered,
    batst_set_indexes=batst_set_filtered_indexes,
    im2age_map_batst=im2age_map_test,
    min_age=cfg.MIN_AGE,
    age_interval=cfg.AGE_INTERVAL,
    max_age=cfg.MAX_AGE,
    transform=transf,
    copies=1, 
    age_radius=cfg.APREF_VLD_SET_AGE_DIFF_LEARN_RADIUS_HI, # this should be the radius of actual learning in order to give correct labels. Not the absolute AGE_RADIUS
    embs_trn=face2emb_arr_trn_r,
    embs_vld=face2emb_arr_vld_r,
    num_references=cfg.NUM_OF_REFS,
    use_embs=cfg.USE_EMBEDDINGS,
    embs_knn=cfg.EMBEDDINGS_KNN
)

print("Testing (q tst where AgePredict(q)  {age_diff_learn_radius_lo} <= error <= {age_diff_learn_radius_hi}, r trn where age(r)=AgePredict(q)) set size: ".format(age_diff_learn_radius_lo=cfg.AGE_DIFF_LEARN_RADIUS_LO, age_diff_learn_radius_hi=cfg.AGE_DIFF_LEARN_RADIUS_HI) + str(len(test_apref_ds)))

x_test_all, y_test_all, batst_set_all_indexes = get_error_constrained_dataset(orig_dataset_images=x_test, 
                                                                                                orig_dataset_metadata=y_test,
                                                                                                age_diff_learn_radius_lo=0,
                                                                                                age_diff_learn_radius_hi=35,#cfg.AGE_RADIUS,
                                                                                                im2age_map_batst=im2age_map_test)


# apref = Age Predict Reference (dataset is based on (q,r) pairs where r's age is what AgePredict model returns on q)
# test_apref_all_ds = DiffPipeline4MimicDiffDataset(
# 	batrn_set_images=x_train,
# 	batrn_set_metadata=y_train,
#     batst_set_images=x_test_all,
#     batst_set_metadata=y_test_all,
#     batst_set_indexes=batst_set_all_indexes,
#     im2age_map_batst=im2age_map_test,
# 	min_age=cfg.MIN_AGE,
# 	age_interval=cfg.AGE_INTERVAL,
#     max_age=cfg.MAX_AGE,
# 	transform=transf,
#     copies=1, 
#     age_radius=cfg.AGE_RADIUS,
#     embs_trn=face2emb_arr_trn_r,
#     embs_vld=face2emb_arr_vld_r#,
#     #num_references=cfg.NUM_OF_REFS
# )

# print("Testing (q tst where AgePredict(q)  {age_diff_learn_radius_lo} <= error <= {age_diff_learn_radius_hi}, r trn where age(r)=AgePredict(q)) set size: ".format(age_diff_learn_radius_lo=cfg.AGE_DIFF_LEARN_RADIUS_LO, age_diff_learn_radius_hi=35) + str(len(test_apref_all_ds))) #cfg.AGE_RADIUS





image_datasets = {
    'train': train_ds,
    'val_qtst_rtst': test_qtst_rtst_ds,
    'val_qtst_rtrn': test_qtst_rtrn_ds,
    'val_apref_ds': test_apref_ds,
    #'val_apref_all_ds': test_apref_all_ds
}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val_qtst_rtst', 'val_qtst_rtrn', 'val_apref_ds']}#, 'val_apref_all_ds']}

data_loaders = {
    'train': DataLoader(train_ds, batch_size=cfg.BATCH_SIZE, num_workers=cfg.NUM_OF_WORKERS_DATALOADER, pin_memory=True, shuffle=True, drop_last=True),
    'val_qtst_rtst': DataLoader(test_qtst_rtst_ds, batch_size=cfg.BATCH_SIZE, num_workers=cfg.NUM_OF_WORKERS_DATALOADER, pin_memory=True, shuffle=False, drop_last=True),
    'val_qtst_rtrn': DataLoader(test_qtst_rtrn_ds, batch_size=cfg.BATCH_SIZE, num_workers=cfg.NUM_OF_WORKERS_DATALOADER, pin_memory=True, shuffle=False, drop_last=True),
    'val_apref_ds': DataLoader(test_apref_ds, batch_size=cfg.BATCH_SIZE, num_workers=cfg.NUM_OF_WORKERS_DATALOADER, pin_memory=True, shuffle=False, drop_last=True),
    #'val_apref_all_ds' : DataLoader(test_apref_all_ds, batch_size=cfg.BATCH_SIZE, num_workers=cfg.NUM_OF_WORKERS_DATALOADER, pin_memory=True, shuffle=False, drop_last=True)
}

C:\Users\proj\.conda\envs\my_thesis_py38\lib\site-packages\torchvision\transforms\transforms.py:1361: UserWarning: Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead
  warnings.warn(


Training (q trn, r trn) set size: 44285
Testing (q tst, r tst) set size: 10606
Testing (q tst, r trn) set size: 10606
Testing (q tst where AgePredict(q)  0 <= error <= 3, r trn where age(r)=AgePredict(q)) set size: 7012


In [5]:
# for i, batch in enumerate(tqdm(data_loaders['train'])):
#     if i == 100:
#         break

In [6]:
# import matplotlib.pyplot as plt
# total_diffs, len_of_potential_refs = train_ds.get_stats()
# plt.hist(np.array(total_diffs))

In [7]:
# plt.hist(np.array(len_of_potential_refs)) #, bins=100)

In [8]:
# total_diffs

In [9]:
# for i, batch in enumerate(tqdm(data_loaders['val_qtst_rtst'])):
#     if i == 100:
#         break

In [10]:
# for i, batch in enumerate(tqdm(data_loaders['val_qtst_rtrn'])):
#     if i == 100:
#         break

In [12]:
# for i, batch in enumerate(tqdm(data_loaders['val_apref_ds'])):
#     if i == 100:
#         break

In [ ]:
total_diffs_v, len_of_potential_refs_v = test_apref_ds.get_stats()
plt.hist(np.array(total_diffs_v))

In [ ]:
plt.hist(np.array(len_of_potential_refs_v)) #, bins=100)

In [ ]:
# len_of_potential_refs_v_arr = np.array(len_of_potential_refs_v)
# len_of_potential_refs_v_arr[len_of_potential_refs_v_arr < 200]